In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_243131/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 10:10:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 10:10:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/12 10:10:06 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
data = [[3, 'Brad', None, 4000], [1, 'John', 3, 1000], [2, 'Dan', 3, 2000], [4, 'Thomas', 3, 4000]]
employee = pd.DataFrame(data, columns=['empId', 'name', 'supervisor', 'salary']).astype({'empId':'Int64', 'name':'object', 'supervisor':'Int64', 'salary':'Int64'})
data = [[2, 500], [4, 2000]]
bonus = pd.DataFrame(data, columns=['empId', 'bonus']).astype({'empId':'Int64', 'bonus':'Int64'})


df_person = spark.createDataFrame(employee)
df_person.createOrReplaceTempView("Employee")

df_person = spark.createDataFrame(bonus)
df_person.createOrReplaceTempView("Bonus")



In [5]:
employee

,empId,name,supervisor,salary
0,3,Brad,<NA>,4000
1,1,John,3,1000
2,2,Dan,3,2000
3,4,Thomas,3,4000


In [6]:
bonus

,empId,bonus
0,2,500
1,4,2000


In [7]:
query = """
    select
        e.name as name,
        b.bonus as bonus
    from
        Employee e left join Bonus b on e.empId = b.empId
    where
        b.bonus is null or b.bonus < 1000
"""

In [8]:
spark.sql(query).show()

+----+-----+
|name|bonus|
+----+-----+
| Dan|  500|
|John| NULL|
|Brad| NULL|
+----+-----+

